In [42]:
# comando de Jupyter para que las imagenes se muestren automaticamente 
%matplotlib inline

import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

# Carga Datos

In [43]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [44]:
# summarize loaded dataset
print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))

Train: X=(50000, 32, 32, 3), y=(50000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)


In [45]:
n,h,w,c = x_train.shape
classes=int(y_train.max()+1)

# Aumento de clases

In [46]:
#datagen = ImageDataGenerator(rotation_range=20,
#                             rescale=1./255,
#                             horizontal_flip=True,
#                             )

In [47]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [48]:
# Aca hacerlo por canal
for i in range(c):
    x_train = (x_train - x_train.mean()) / x_train.std()
    x_test = (x_test - x_test.mean()) / x_test.std()

In [49]:
generator = datagen.flow(x_train, y_train, batch_size=32)

In [50]:
input_shape = (32, 32, 3)

## Modelo 1

In [51]:
es = EarlyStopping(monitor='val_acc', mode='min', patience=10, verbose=1)

model = Sequential()
model.add(Conv2D(64, kernel_size = 7, activation='relu', input_shape=(input_shape), strides = 1))
model.add(MaxPooling2D(pool_size=(2,2), strides = 2))
model.add(Flatten())
model.add(Dense(classes, activation='softmax'))

model.compile(
  optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,amsgrad=False), 
  loss='sparse_categorical_crossentropy', 
  # metricas para ir calculando en cada iteracion o batch 
  # Agregamos el accuracy del modelo
  metrics=['accuracy'], 
)

In [52]:
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 64)        9472      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                108170    
Total params: 117,642
Trainable params: 117,642
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
model.fit_generator(generator, steps_per_epoch=len(generator)/32, epochs=100, validation_data=(x_test,y_test), callbacks=[es])

Epoch 1/100


C:\Users\dalessam\AppData\Local\Continuum\anaconda3\envs\deep_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\dalessam\AppData\Local\Continuum\anaconda3\envs\deep_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


49/48 [==============================] - 4s 86ms/step - loss: 2.1408 - acc: 0.2270 - val_loss: 1.9105 - val_acc: 0.2987
Epoch 2/100
49/48 [==============================] - 3s 69ms/step - loss: 1.9630 - acc: 0.2876 - val_loss: 1.8251 - val_acc: 0.3476
Epoch 3/100
49/48 [==============================] - 3s 65ms/step - loss: 1.9523 - acc: 0.3099 - val_loss: 1.8233 - val_acc: 0.3588
Epoch 4/100
49/48 [==============================] - 3s 66ms/step - loss: 1.9363 - acc: 0.3023 - val_loss: 1.7866 - val_acc: 0.3695
Epoch 5/100
49/48 [==============================] - 3s 71ms/step - loss: 1.9396 - acc: 0.3176 - val_loss: 1.7557 - val_acc: 0.3667
Epoch 6/100
49/48 [==============================] - 3s 69ms/step - loss: 1.8859 - acc: 0.3202 - val_loss: 1.6565 - val_acc: 0.4126
Epoch 7/100
49/48 [==============================] - 4s 72ms/step - loss: 1.8095 - acc: 0.3457 - val_loss: 1.7282 - val_acc: 0.3990
Epoch 8/100
49/48 [==============================] - 3s 68ms/step - loss: 1.8412 - acc: 

In [54]:
test_generator = datagen.flow(x_test,y_test,batch_size=32)
model.evaluate_generator(test_generator)

[1.7867029775619507, 0.3673]

## Modelo 2

In [57]:
es = EarlyStopping(monitor='val_acc', mode='min', patience=10, verbose=1)

conv_filters=32
dense_filters=64

model = Sequential()
model.add(Conv2D(conv_filters,(3,3),activation=None,input_shape=input_shape))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dense(dense_filters,activation=None))
model.add(Activation("relu"))
model.add(Dense(classes,activation="softmax"))


model.compile(
  optimizer=keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,amsgrad=False), 
  loss='sparse_categorical_crossentropy', 
  # metricas para ir calculando en cada iteracion o batch 
  # Agregamos el accuracy del modelo
  metrics=['accuracy'], 
)

In [58]:
model.fit_generator(generator, steps_per_epoch=len(generator)/32, epochs=100, validation_data=(x_test,y_test), callbacks=[es])

Epoch 1/100


C:\Users\dalessam\AppData\Local\Continuum\anaconda3\envs\deep_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\dalessam\AppData\Local\Continuum\anaconda3\envs\deep_env\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


49/48 [==============================] - 5s 108ms/step - loss: 2.2745 - acc: 0.1958 - val_loss: 2.0248 - val_acc: 0.2672
Epoch 2/100
49/48 [==============================] - 4s 77ms/step - loss: 2.0063 - acc: 0.2793 - val_loss: 1.8787 - val_acc: 0.3154
Epoch 3/100
49/48 [==============================] - 4s 75ms/step - loss: 1.9408 - acc: 0.3017 - val_loss: 1.8152 - val_acc: 0.3417
Epoch 4/100
49/48 [==============================] - 4s 72ms/step - loss: 1.9261 - acc: 0.2946 - val_loss: 1.7628 - val_acc: 0.3553
Epoch 5/100
49/48 [==============================] - 4s 78ms/step - loss: 1.8625 - acc: 0.3202 - val_loss: 1.7327 - val_acc: 0.3718
Epoch 6/100
49/48 [==============================] - 4s 75ms/step - loss: 1.7927 - acc: 0.3450 - val_loss: 1.7517 - val_acc: 0.3699
Epoch 7/100
49/48 [==============================] - 4s 73ms/step - loss: 1.7649 - acc: 0.3533 - val_loss: 1.7010 - val_acc: 0.3831
Epoch 8/100
49/48 [==============================] - 3s 69ms/step - loss: 1.7681 - acc:

In [59]:
test_generator = datagen.flow(x_test,y_test,batch_size=32)
model.evaluate_generator(test_generator)

[1.6818044284820557, 0.3846]